In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

import re

from IPython.display import clear_output

In [2]:

data = pd.read_csv('population1.txt', sep="\t", header=5)


In [3]:
head= pd.read_csv('population1.txt', sep="\t")
head = head.head(10)

In [4]:
head[['Unnamed: 22','Unnamed: 23','Unnamed: 24']]

,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,NaN,NaN,NaN
1,NaN,No,NaN
2,First,significant,FIPS
3,census,change since,code
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,1790,1900,NaN
7,NaN,NaN,NaN
8,1800,1820,01
9,1880,1880,02


In [5]:
data = data.rename(columns={'Unnamed: 0':'state','Unnamed: 22': "first_census", 'Unnamed: 23': "no_sig_change_since", 'Unnamed: 24':'fips_code'})

In [6]:
data.head()

,state,1990,1980,1970,1960,1950,1940,1930,1920,1910,...,1850,1840,1830,1820,1810,1800,1790,first_census,no_sig_change_since,fips_code
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UNITED STATES,"248,709,873","226,545,805","203,211,926","179,323,175","151,325,798","132,164,569","123,202,624","106,021,537","92,228,496",...,"23,191,876","17,063,353","12,860,702","9,638,453","7,239,881","5,308,483","3,929,214",1790.0,1900.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alabama,"4,040,587","3,893,888","3,444,165","3,266,740","3,061,743","2,832,961","2,646,248","2,348,174","2,138,093",...,"771,623","590,756","309,527","127,901","9,046","1,250",---,1800.0,1820.0,1.0
4,Alaska,"550,043","401,851","300,382","226,167","128,643","72,524","59,278","55,036","64,356",...,---,---,---,---,---,---,---,1880.0,1880.0,2.0


In [7]:
data.columns

Index(['state', '1990', '1980', '1970', '1960', '1950', '1940', '1930', '1920',
       '1910', '1900', '1890', '1880', '1870', '1860', '1850', '1840', '1830',
       '1820', '1810', '1800', '1790', 'first_census', 'no_sig_change_since',
       'fips_code'],
      dtype='object')

In [19]:
data[['state','1870','1880','1890','1900']].dropna(axis=0).to_csv('states_population.csv')

In [9]:
def parse_urban_lines(l):
    result = []
    
    for x in l:
        

        try:
#            x = x.replace('*','')
            x = re.sub('\.\.+','---',x)


            x = re.sub('\-\-+','', x)
        
            s = x.split('   ')
            if len(s) == 3:
                result.append(s[0:3])
            elif len(s) == 6:
                result.append(s[0:3])
                result.append(s[3:6])
            elif len(s) == 4:
                result.append(s.remove(''))
            elif len(s) == 2:
                result.append(x.split('  '))
                
            else:
#                pass
                result.append((str(s), np.nan,np.nan))
                
        except:
            result.append([x,'blank'])
    return result


In [10]:
years = ['1870', '1880','1890','1900']

In [11]:
cities_ = []

for year in years:
    cities_raw = pd.read_csv('cities_{0}.txt'.format(year), sep="      ",header=4, engine='python')

    try:
        cities1 = cities_raw['Rank | Place']
    except:
        cities1 = cities_raw['Rank | Place 1/']

    cities2 = cities_raw['Unnamed: 1']


    cities1_list = cities1.dropna().tolist()
    cities2_list = cities2.dropna().tolist()

    cities_list = parse_urban_lines(cities1_list) + parse_urban_lines(cities2_list)
    cities_.append(cities_list)


In [34]:
cities_dict = {}
    
for i in range(len(years)):
    year = years[i]
    cities_list = cities_[i]
    cities = pd.DataFrame(cities_list, columns=['rank','region','Population'])
    cities = cities.dropna(axis=0)
    
    cities['state'] = cities.region.apply(lambda x:re.findall('\s[A-Z][A-Z]\s*',x)[0].strip())
    cities['city'] = cities.region.apply(lambda x: x.split(re.findall('\s[A-Z][A-Z]\s*',x)[0])[0].strip(','))

    
    cities['Population'] = cities.Population.apply(lambda x: int(''.join(x.split(','))))
    cities = cities.sort_values(by='Population', ascending=False)
    cities = cities.reset_index()
    cities = cities.drop('index',axis=1)
    cities = cities.drop(['rank','region'], axis=1)
    
    print(cities)
    cities_dict[year] = cities
    cities.to_csv('cities_population_{0}.csv'.format(year))
    

    Population state                city
0       942292    NY       New York city
1       674022    PA   Philadelphia city
2       396099    NY       Brooklyn city
3       310864    MO      St. Louis city
4       298977    IL        Chicago city
..         ...   ...                 ...
95       15389    GA        Augusta city
96       15357    NY         Cohoes city
97       15087    KY        Newport city
98       15058    NJ  New Brunswick city
99       14930    IA     Burlington city

[100 rows x 3 columns]
    Population state               city
0      1206299    NY      New York city
1       847170    PA  Philadelphia city
2       566663    NY      Brooklyn city
3       503185    IL       Chicago city
4       362839    MA        Boston city
..         ...   ...                ...
95       20550    TX   San Antonio city
96       20541    NY        Elmira city
97       20433    KY       Newport city
98       20207    NY  Poughkeepsie city
99       19743    IL   Springfield city

[10

In [20]:
#cities_dict